In [1]:
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# matplotlib 설정
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# GPU를 사용할 경우
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """transition 저장"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [3]:
from torchvision import models

class DQN(nn.Module):

    def __init__(self, n_observations, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, n_actions)

    # 최적화 중에 다음 행동을 결정하기 위해서 하나의 요소 또는 배치를 이용해 호촐됩니다.
    # ([[left0exp,right0exp]...]) 를 반환합니다.
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)

In [4]:
import numpy as np

# BATCH_SIZE는 리플레이 버퍼에서 샘플링된 트랜지션의 수입니다.
# GAMMA는 이전 섹션에서 언급한 할인 계수입니다.
# EPS_START는 엡실론의 시작 값입니다.
# EPS_END는 엡실론의 최종 값입니다.
# EPS_DECAY는 엡실론의 지수 감쇠(exponential decay) 속도 제어하며, 높을수록 감쇠 속도가 느립니다.
# TAU는 목표 네트워크의 업데이트 속도입니다.
# LR은 ``AdamW`` 옵티마이저의 학습율(learning rate)입니다.
BATCH_SIZE = 128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# gym 행동 공간에서 행동의 숫자를 얻습니다.
n_actions = 3
n_history = 60

policy_net = models.efficientnet_b0().to(device)
target_net = models.efficientnet_b0().to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return policy_net(state).max(1).indices.view(1, 1)
    else:
        return torch.tensor([[np.random.choice([a for a in range(n_actions)])]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations(show_result=False):
    plt.figure(1)
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # 100개의 에피소드 평균을 가져 와서 도표 그리기
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # 도표가 업데이트되도록 잠시 멈춤
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())

In [5]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). 이것은 batch-array의 Transitions을 Transition의 batch-arrays로
    # 전환합니다.
    batch = Transition(*zip(*transitions))

    # 최종이 아닌 상태의 마스크를 계산하고 배치 요소를 연결합니다
    # (최종 상태는 시뮬레이션이 종료 된 이후의 상태)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Q(s_t, a) 계산 - 모델이 Q(s_t)를 계산하고, 취한 행동의 열을 선택합니다.
    # 이들은 policy_net에 따라 각 배치 상태에 대해 선택된 행동입니다.
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # 모든 다음 상태를 위한 V(s_{t+1}) 계산
    # non_final_next_states의 행동들에 대한 기대값은 "이전" target_net을 기반으로 계산됩니다.
    # max(1).values로 최고의 보상을 선택하십시오.
    # 이것은 마스크를 기반으로 병합되어 기대 상태 값을 갖거나 상태가 최종인 경우 0을 갖습니다.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1).values
    # 기대 Q 값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Huber 손실 계산
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    # 변화도 클리핑 바꿔치기
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), 100)
    optimizer.step()

In [6]:
from collections import deque
import numpy as np
from ata.exchange.offlineexchangesimulator import OfflineExchangeSimulator
from ata.utils.trade import *

def min_max_normalize(x):
    if np.max(x) == np.min(x):  # 모든 값이 같으면 0으로 처리
        return np.zeros_like(x)
    return (x - np.min(x)) / (np.max(x) - np.min(x))

def z_score_normalize(x):
    mean = np.mean(x)
    std = np.std(x)
    if std == 0:
        return np.zeros_like(x)
    return (x - mean) / std

def log_transform(x):
    return np.log1p(x)  # log(1 + x)
    
def normalize_balance(x):
    x = log_transform(x)  # 로그 변환 적용
    return z_score_normalize(x)  # Z-score 정규화 적용

def to_three_channels(single_channel_data):
    # 단일 채널 데이터가 (H, W) 형태라고 가정
    three_channel_data = np.stack([single_channel_data] * 3, axis=-1)  # (H, W, 3)
    
    # (H, W, 3) -> (3, H, W) 형태로 변환
    three_channel_data = np.transpose(three_channel_data, (2, 0, 1))  # (3, H, W)
    
    return three_channel_data

def insert_balance_data(exchange: OfflineExchangeSimulator, balance_history: dict):
    balance_history['total_balance'].append(exchange.get_total_balance())
    balance_history['total_krw'].append(exchange.balance['KRW']['total'])
    balance_history['free_krw'].append(exchange.balance['KRW']['free'])
    balance_history['used_krw'].append(exchange.balance['KRW']['used'])
    balance_history['total_item'].append(exchange.balance['BTC']['total'])
    balance_history['free_item'].append(exchange.balance['BTC']['free'])
    balance_history['used_item'].append(exchange.balance['BTC']['used'])
    return balance_history

def preprocess_state(ohlcv_1m:dict, balance_history:dict, device):
    ohlcv_1m, keys = calc_bollinger_bands(ohlcv_1m, 20, 2)
    upper_key = keys['upper_key']
    lower_key = keys['lower_key']
    b_key = keys['b_key']
    ohlcv_1m, mfi_key = calc_mfi(ohlcv_1m, 14)
    state = []
    state.append(z_score_normalize(ohlcv_1m['open'].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m['high'].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m['low'].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m['close'].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m['volume'].iloc[-n_history:]))
    
    state.append(z_score_normalize(ohlcv_1m[upper_key].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m[lower_key].iloc[-n_history:]))
    state.append(z_score_normalize(ohlcv_1m[b_key].iloc[-n_history:]))
    state.append(min_max_normalize(ohlcv_1m[mfi_key].iloc[-n_history:]))
    
    for key in balance_history:
        state.append(normalize_balance(list(balance_history[key])[-n_history:]))
    
    state = np.array(state)
    state = state.T
    return torch.tensor(to_three_channels(state), dtype=torch.float32).to(device).unsqueeze(0)
    

In [ ]:
import sys

file_paths = ['jup_30000.csv', 'BTC_Data.csv']

for file_path in file_paths:
    print(file_path)
    exchange = OfflineExchangeSimulator(file_path=file_path, balance=1000000)
    exchange.init()
    balance_history = {}
    balance_history['total_balance'] = deque([exchange.get_total_balance() for _ in range(n_history)], maxlen=n_history)
    balance_history['total_krw'] = deque([exchange.balance['KRW']['total'] for _ in range(n_history)], maxlen=n_history)
    balance_history['free_krw'] = deque([exchange.balance['KRW']['free'] for _ in range(n_history)], maxlen=n_history)
    balance_history['used_krw'] = deque([exchange.balance['KRW']['used'] for _ in range(n_history)], maxlen=n_history)
    balance_history['total_item'] = deque([exchange.balance['BTC']['total'] for _ in range(n_history)], maxlen=n_history)
    balance_history['free_item'] = deque([exchange.balance['BTC']['free'] for _ in range(n_history)], maxlen=n_history)
    balance_history['used_item'] = deque([exchange.balance['BTC']['used'] for _ in range(n_history)], maxlen=n_history)
    ohlcv_1m = exchange.get_ohlcv_per_1m('BTC')
    state = preprocess_state(ohlcv_1m, balance_history, device)
    for t in count():
        action = select_action(state)
        reward = 0
        if action == 1 and exchange.balance['KRW']['free'] * 0.9499999 > 6000:
            buy_krw = exchange.balance['KRW']['free']
            order_id = exchange.create_buy_order_at_market_price('BTC', exchange.balance['KRW']['free'] * 0.9499999)
            buy_price = exchange.get_order(order_id)['price']
        elif action == 2 and exchange.balance['BTC']['free'] > 0:
            order_id = exchange.create_sell_order_at_market_price('BTC', exchange.balance['BTC']['free'])
            order = exchange.get_order(order_id)
            reward = order['price'] * order['amount'] / buy_krw * 100
            buy_krw = buy_price = 0
        reward = torch.tensor([reward], dtype=torch.float32).to(device)
        done = not exchange.update()
        if done:
            next_state = None
        else:
            ohlcv_1m = exchange.get_ohlcv_per_1m('BTC')
            balance_history = insert_balance_data(exchange, balance_history)
            next_state = preprocess_state(ohlcv_1m, balance_history, device)
            
        memory.push(state, action, next_state, reward)
        state = next_state
        optimize_model()

        # 목표 네트워크의 가중치를 소프트 업데이트
        # θ′ ← τ θ + (1 −τ )θ′
        target_net_state_dict = target_net.state_dict()
        policy_net_state_dict = policy_net.state_dict()
        for key in policy_net_state_dict:
            target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
        target_net.load_state_dict(target_net_state_dict)
        sys.stdout.write(f"\r{t:<5}: total: {exchange.get_total_balance()}              ")
        if done:
            episode_durations.append(exchange.get_total_balance())
            plot_durations()
            print()
            break
        
        
        
print('Complete')
plot_durations(show_result=True)
plt.ioff()
plt.show()

<Figure size 640x480 with 0 Axes>